In [2]:
import os

In [3]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Clone Github repositories

In [4]:
%pwd

'e:\\Amarnath\\Mastering-GENAI\\18. Mar30th Code Analysis in GEN AI\\Code_Helper_Analysis_GenAI\\research'

In [6]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [7]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/amarnath2706/CircleCI_Docker", to_path=repo_path)

<git.repo.base.Repo 'e:\\Amarnath\\Mastering-GENAI\\18. Mar30th Code Analysis in GEN AI\\Code_Helper_Analysis_GenAI\\research\\test_repo\\.git'>

In [10]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path,
                                       

                                       
#loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [11]:
documents = loader.load()

In [9]:
documents

[Document(page_content='import os\nimport sys\nimport logging\n\nlogging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"\n\nlog_dir = "logs"\nlog_filepath = os.path.join(log_dir,"running_logs.log")\nos.makedirs(log_dir, exist_ok=True)\n\n\nlogging.basicConfig(\n    level= logging.INFO,\n    format= logging_str,\n\n    handlers=[\n        logging.FileHandler(log_filepath),\n        logging.StreamHandler(sys.stdout)\n    ]\n)\n\nlogger = logging.getLogger("mlProjectLogger")', metadata={'source': 'test_repo\\src\\mlProject\\__init__.py', 'language': <Language.PYTHON: 'python'>}),
 Document(page_content='import os\nimport urllib.request as request\nimport zipfile\nfrom mlProject import logger\nfrom mlProject.utils.common import get_size\nfrom mlProject.entity.config_entity import DataIngestionConfig\nfrom pathlib import Path\n\n\nclass DataIngestion:\n    def __init__(self, config: DataIngestionConfig):\n        self.config = config\n\n    \n    def download_file(self):\n    

### Chunkings

In [12]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [13]:
texts = documents_splitter.split_documents(documents)

In [14]:
len(texts)

3

### Embedding model

In [15]:
os.environ["OPENAI_API_KEY"] = "********************************************"

In [16]:
embeddings=OpenAIEmbeddings(disallowed_special=())

### Knowledge base (vector DB)

In [17]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

### LLM Wrapper

In [18]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

In [19]:
#define the buffer memory
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [20]:
#Ques and ans 
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

### Q&A

In [27]:
question = "app.route"

In [28]:
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 9, updating n_results = 9
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-NgVDPOlvY0sJaQtDjzC2DoLo on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-NgVDPOlvY0sJaQtDjzC2DoLo on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increa

KeyboardInterrupt: 